In [655]:
MODEL_TYPE = 'facebook/bart-base'

# Data

In [656]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [657]:
data = pd.read_json('data/etude.json')
data.head()

,source,decompiled
0,interface A {\n fun funFromA() {}\n\n}\n,public interface A {\n\n \n public stati...
1,"class Address {\n var name: String = ""Holme...",public final class Address {\n\n \n publ...
2,object AnotherGame {\n\n}\n\n,public final class AnotherGame {\n public s...
3,class ArticleHeadline {\n\n}\n,public final class ArticleHeadline {\n}\n
4,"enum class Bit {\n ZERO, ONE\n}","public enum Bit {\n ZERO,\n ONE\n}\n"


# Tokenization

In [658]:
from transformers import BartTokenizer

In [659]:
tokenizer = BartTokenizer.from_pretrained(MODEL_TYPE)
tokenizer.padding = "max_length"
tokenizer.truncation = True
"""
TODO: Нужно обеспечить установку параметров токенизации
    padding="max_length"
    truncation=True
"""

loading file vocab.json from cache at /Users/eshevlyakov/.cache/huggingface/hub/models--facebook--bart-base/snapshots/aadd2ab0ae0c8268c7c9693540e9904811f36177/vocab.json
loading file merges.txt from cache at /Users/eshevlyakov/.cache/huggingface/hub/models--facebook--bart-base/snapshots/aadd2ab0ae0c8268c7c9693540e9904811f36177/merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /Users/eshevlyakov/.cache/huggingface/hub/models--facebook--bart-base/snapshots/aadd2ab0ae0c8268c7c9693540e9904811f36177/config.json
Model config BartConfig {
  "_name_or_path": "facebook/bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "clas

'\nTODO: Нужно обеспечить установку параметров токенизации\n    padding="max_length"\n    truncation=True\n'

# Data Collator

In [660]:
from transformers import DataCollatorForSeq2Seq

In [661]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=MODEL_TYPE)

# Evaluate

In [662]:
import evaluate
import numpy as np
from transformers import EvalPrediction

In [663]:
metric = evaluate.load("sacrebleu")

In [664]:
def compute_metrics(eval_preds: EvalPrediction) -> dict|None:
    # References List[] -> List[str]
    refs = eval_preds.label_ids
    refs = np.where(refs != -100, refs, tokenizer.pad_token_id) # TODO: why refs != -100
    refs = tokenizer.batch_decode(refs)
    refs = [[ref.strip()] for ref in refs]

    # Predictions List[int] -> List[str]
    preds = eval_preds.predictions
    if isinstance(preds, tuple):
        preds = preds[0]
    preds = tokenizer.batch_decode(preds)
    preds = [pred.strip() for pred in preds]

    # Metric bleu
    result = metric.compute(references=refs, predictions=preds)
    result = {"bleu": result["score"]}

    # Metric gen_len
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}

    return result

# Train the model

In [665]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, PreTrainedTokenizer
from datasets import Dataset

Form data

In [666]:
def data_to_datasets(data: pd.DataFrame, tokenizer: PreTrainedTokenizer, test_size=0.2):
    train_data, test_data = train_test_split(data, test_size=test_size)

    train_dataset = Dataset.from_pandas(train_data)
    test_dataset = Dataset.from_pandas(test_data)

    tokenize = lambda x: tokenizer(
        text=x["decompiled"],
        text_target=x["source"],
        padding="max_length",
        truncation=True
    )

    tokenized_train_dataset = train_dataset.map(tokenize)
    tokenized_test_dataset = test_dataset.map(tokenize)

    return tokenized_train_dataset, tokenized_test_dataset

In [667]:
train_dataset, test_dataset = data_to_datasets(data, tokenizer)

Map:   0%|          | 0/324 [00:00<?, ? examples/s]

Map:   0%|          | 0/81 [00:00<?, ? examples/s]

Model

In [668]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_TYPE)
"""
TODO: подточить под архитектуру тачки
"""

loading configuration file config.json from cache at /Users/eshevlyakov/.cache/huggingface/hub/models--facebook--bart-base/snapshots/aadd2ab0ae0c8268c7c9693540e9904811f36177/config.json
Model config BartConfig {
  "_name_or_path": "facebook/bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL

'\nTODO: подточить под архитектуру тачки\n'

Training args

In [669]:
model_name = 'first_BART_model'

training_args = Seq2SeqTrainingArguments(
    output_dir=model_name,
    evaluation_strategy="epoch",
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_total_limit=3,
    predict_with_generate=True,
    # Whether to use generate to calculate generative metrics (ROUGE, BLEU).
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Trainer

In [670]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Build the model

In [671]:
result = trainer.train()

The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: decompiled, __index_level_0__, source. If decompiled, __index_level_0__, source are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
/Users/eshevlyakov/anaconda3/envs/deobfuscator/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 324
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 123
  Number of trainable parameters = 139420416


Epoch,Training Loss,Validation Loss


The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: decompiled, __index_level_0__, source. If decompiled, __index_level_0__, source are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 81
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: decompiled, __index_level_0__, source. If decompiled, __index_level_0__, source are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 81
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: decompiled, __index_level_0__, source. If decompiled, __index_level_0__, so

In [672]:
trainer.save_model()

Saving model checkpoint to first_BART_model
Configuration saved in first_BART_model/config.json
Model weights saved in first_BART_model/pytorch_model.bin
tokenizer config file saved in first_BART_model/tokenizer_config.json
Special tokens file saved in first_BART_model/special_tokens_map.json
